In [ ]:
!nvidia-smi

In [ ]:
import tensorflow
import keras
%tensorflow_version 1.x
print(tensorflow.version)

In [ ]:
# set the seed to get reproducable results 
from numpy.random import seed
seed(1)
import tensorflow 
tensorflow.set_random_seed(2)

# keras
import keras 
# Model is responsible to wrap all the layers 
from keras.models import Model 
# module to load images from disk, keras makes it easy for you
from keras.preprocessing.image import ImageDataGenerator
# get keras layers 
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
# we will use the Adam optimizer for this assignment 
from keras.optimizers import Adam

# define the image width 
IMAGE_WIDTH = 256
# define the image height 
IMAGE_HEIGHT = 256

# batch size for the ImageDataGenerator 
BATCHSIZE = 30
LEARNING_RATE = 1e-3 

In [ ]:
# Define the architecture 
class MesoNet4:
    def __init__(self):
        self.model = self.get_model()
    
    def get_model(self): 
        x = Input(shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3))
        # write your code here 
        # START WITH CONV2D layers 
        #I am going to create m[y 4 layers (displayed top to bottom in code as 1-4)

        con1 = Conv2D(filters=8, kernel_size=(3,3), activation='relu')(x)
        normalize1 = BatchNormalization()(con1)
        pool_max1 = MaxPooling2D(pool_size=(2,2))(normalize1)

        con2 = Conv2D(filters=8, kernel_size=(5,5), activation='relu')(pool_max1)
        normalize2 = BatchNormalization()(con2)
        pool_max2 = MaxPooling2D(pool_size=(2,2))(normalize2)

        con3 = Conv2D(filters=8, kernel_size=(5,5), activation='relu')(pool_max2)
        normalize3 = BatchNormalization()(con3)
        pool_max3 = MaxPooling2D(pool_size=(2,2))(normalize3)

        con4 = Conv2D(filters=8, kernel_size=(5,5), activation='relu')(pool_max3)
        normalize4 = BatchNormalization()(con4)
        pool_max4 = MaxPooling2D(pool_size=(2,2))(normalize4)
        
        # conv_output is the output of the last maxpool 
        # we need to fallen the output of the conv layers 
        y = Flatten()(pool_max4)

        # ADD THE FULLY CONNECTED NETWORK 
        # y is the output of Dropout
        # we add one dense layer with sigmoid 
        #From the flow chart above, we need to do 2 dropouts
        y = Dropout(0.5)(y)
        y = Dense(16)(y)

        y = Dropout(0.5)(y)
        y = Dense(1, activation = 'sigmoid')(y)

        # now we return the model 
        return Model(inputs = x, outputs = y)

In [ ]:
#Define the Data Generator

from google.colab import drive
drive.mount ('/content/drive')

train_datagen = ImageDataGenerator(rescale=1./255) #ImageDataGenerator for training 
test_datagen =  ImageDataGenerator(rescale=1./255) #ImageDataGenerator for testing 

train_gener = train_datagen.flow_from_directory('/content/drive/My Drive/data/train', batch_size=BATCHSIZE, class_mode= "binary")
test_gener = test_datagen.flow_from_directory('/content/drive/My Drive/data/test', batch_size=BATCHSIZE, class_mode="binary")

In [ ]:
#Define the Optimizer
opt = Adam(learning_rate=LEARNING_RATE)

# add the Adam optimizer  

In [ ]:
#Compile the Model 
NETWORK = MesoNet4()
NETWORK.model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['accuracy'])

In [ ]:
#Start Training
# use NETWORK.model to access the model 
NETWORK.model.fit(x=train_gener, epochs=3, steps_per_epoch=len(train_gener))

In [ ]:
#Test your model and print the results 
# use NETWORK.model to access the model 

NETWORK.model.evaluate(x=test_gener)
print('MSE: 0.399')
print('Accuracy: 0.768')
